## <span style="color: blue;">数据爬取（初学者尝试😩）</span>  

### <span style="color: blue;">1.思路介绍</span>
<p style="text-indent:2em;">本项目爬取的是 https://cn.tripadvisor.com 网页上关于上海市餐厅的一些数据，包括：名称、链接、评分、评价数、类别、人均（以￥表示）和地址。爬取的思路借鉴了 "<strong>下厨房</strong>" 的例子，总体思想为：先通过列表页爬取所有餐厅的名称和每个餐厅具体详情页的链接，然后再利用各餐厅的链接进一步访问餐厅的详情数据： 评分、评价数、类别、人均（以￥表示）和地址。</p>

### <span style="color: blue;">2.过程介绍</span>
<p style="text-indent:2em;">然而，<strong><span style="color: blue;">想法很美好，现实很残酷😢</span></strong>，在一次次失败、IP被封的情况下，艰难的数据爬取过程主要可以分为以下三个阶段:</p>

<p style="text-indent:2em;">
<strong>第一阶段：</strong> 可称为“初次相遇”的<span style="color: red;">友好阶段</span>。在这个阶段，我们先进行一些简单的尝试，来判断爬取的可行性。期间我们多次尝试简单的数据爬取，仅用简单的请求头便可以快速方便地爬出第一页完整的名称数据，没有任何问题，导致我们误认为网页很友好。然后便快速加上了爬取其他数据的代码，为了以防万一还加了cookie，想当然的直接开始了100页（即3000家餐厅）数据的爬取。结果在爬到500家餐厅的时候喜提反爬大礼包，IP封禁一天。代码如下：
</p>

In [ ]:
import requests  # 用于发送 HTTP 请求
from lxml import etree  # 用于解析 HTML
import pandas as pd  # 用于数据处理与保存
import time  # 用于控制爬取时间间隔
import random  # 用于生成随机延时，防止被封禁
import re  # 用于正则表达式匹配

# 基础常量与请求头
BASE_URL = "https://cn.tripadvisor.com"  # TripAdvisor站点基础地址
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 '
                  '(KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36',  # 浏览器标识
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',  # 接收内容类型
    'Accept-Language': 'zh-CN,zh;q=0.9',  # 接收语言
    'Referer': 'https://cn.tripadvisor.com/',  # 来源页
    'Connection': 'keep-alive',  # 长连接
    'Upgrade-Insecure-Requests': '1',  # 安全升级
    'Accept-Encoding': 'gzip, deflate, br',  # 支持压缩格式
}
# 添加 Cookie，替换为有效值以维持会话
headers['cookie'] = 'TAUnique=%1%enc%3AehA2qsyKuYqUNGugE4YEc9dm1eF7t820caB3EeCz%2FJ9SrUNk0jeLYY7X%2FAg52CqdNox8JbUSTxk%3D; TASameSite=1; TASSK=enc%3AAOmzUFPCpffK7LIMOwVntqBefMXOVXI6wkStaBdMhP54mY5qgcvBaHNeF5iZnow%2FsZp4VNaAPbB6FTDolTWcajp7mHRRXpMb7KMOK%2Bh458M9hoYUvNgAPeK2bav99uOsNg%3D%3D; TART=%1%enc%3AC3UfQ0LEIW3NF1z2d%2BxWDqFYetV%2BrK3dlfym18VuqiIy3iJHwE1LpO7n8YRMhaF8mCeVDBnC5k4%3D; TATrkConsent=eyJvdXQiOiJTT0NJQUxfTUVESUEiLCJpbiI6IkFEVixBTkEsRlVOQ1RJT05BTCJ9; _gcl_au=1.1.1498933911.1746629711; _ga=GA1.1.1573501834.1746629711; PMC=V2*MS.33*MD.20250507*LD.20250507; TATravelInfo=V2*A.2*MG.-1*HP.2*FL.3*RS.1; _lc2_fpi=b140173de591--01jtnkwnfyg257s5pzjgyewddd; _lc2_fpi_meta=%7B%22w%22%3A1746629711358%7D; pbjs_sharedId=0a51a26d-e85a-4512-b74b-3e9bab464466; pbjs_sharedId_cst=zix7LPQsHA%3D%3D; _lr_env_src_ats=false; _lr_sampling_rate=100; AMZN-Token=v2FweIB4SFZ1RnhTY1pNZ3dqZ0pUWnlxejVLNjBBdHhPVm5kV1NyNmtoUE43SThNY3VjVkM4ME51SkhKcy9UWG5rR3ZMT0Y2dW0yYUdUZ3FLbE9lMW82NnRVYmVQZEFMbXRuZTIwWHdIc2MxNDFUbjByQjhabDU1MVoyY0k0ckppK29ncWJrdgFiaXZ4IE1SQkg3Nys5RENZRjc3Kzk3Nys5VCsrL3ZlKy92UT09/w==; TAUD=LA-1746629711202-1*RDD-1-2025_05_07*LG-62979-2.1.F.*LD-62980-.....; pbjs_li_nonid=%7B%7D; pbjs_li_nonid_cst=zix7LPQsHA%3D%3D; pbjs_unifiedID=%7B%22TDID%22%3A%22460103ad-102a-418d-a466-288ac8ced98a%22%2C%22TDID_LOOKUP%22%3A%22TRUE%22%2C%22TDID_CREATED_AT%22%3A%222025-04-08T12%3A48%3A55%22%7D; pbjs_unifiedID_cst=zix7LPQsHA%3D%3D; TAAUTHEAT=ikOHzclO9G4Sxe01ABQC3I3IpT7XQpOMdOQw2Q027UVHaYyAwXyUTJx9huNbnp9__TBgEZl5WeJAiUo3socFYvUc066mAF2BbnROkCbl5gy4fWtqAvMCHYAAzd11L8dQSDFfnZYDgXAaQfyTgEiPv3z6Yzxkt0m8KRFQ9F6A4QWjfcmLkrXDKCYIdRqxBVriPJPVoTs2sD8TdbcaBEHhuXDoGauUqZwV7h4; TADCID=sR1Rl4y3l-byrqJLABQCJ4S5rDsRRMescG99HippfoVwdAn9TqGFFdpCvizh8nRIv8sJ8c-NzEeoB_Jah4SOKd0AzomZcGQS7RY; _li_dcdm_c=.tripadvisor.com; TASID=7FF9E722A1057A5FDD2CE407836EB7FE; PAC=AI-Y_UwuP7W9z2RhHDPFtJ0F5QGZDRS6MzayrTd_oqIVJIi_AG3BQtpNht2ohAAGpxVxsBwVaQea2gS3QcSc8PY6VZ61l2-31IqWr9ChSFNt-lTJYXbpcPkZ-aAWDtJtJUUIyR1Fl51auBOwIic95z-gxoGMYQdAMT9zdIXzAr6TSmD_yRNFZpFRizKg6qQvdSIJcOE9AkAY3esFEJiKGncZx2sqvh2VZE7B0N7rVKWr; _lr_retry_request=true; SRT=TART_SYNC; TASession=V2ID.7FF9E722A1057A5FDD2CE407836EB7FE*SQ.27*LS.FindRestaurants*HS.recommended*ES.popularity*DS.5*SAS.popularity*FPS.oldFirst*TS.144F5410E922E3B855ECB902400D4D9C*FA.1*DF.0*TRA.true; OptanonConsent=isGpcEnabled=0&datestamp=Fri+May+09+2025+16%3A33%3A21+GMT%2B0800+(%E4%B8%AD%E5%9B%BD%E6%A0%87%E5%87%86%E6%97%B6%E9%97%B4)&version=202405.2.0&browserGpcFlag=0&isIABGlobal=false&hosts=&consentId=144F5410E922E3B855ECB902400D4D9C&interactionCount=1&isAnonUser=1&landingPath=NotLandingPage&groups=C0001%3A1%2CC0003%3A1%2CSSPD_BG%3A1%2CC0004%3A1%2CC0002%3A1&AwaitingReconsent=false; _gcl_aw=GCL.1746779602.null; __gads=ID=3860aac49c1a1023:T=1746629716:RT=1746779601:S=ALNI_MbYaqBobsrmlfDdKTty6p9HmAUkXQ; __gpi=UID=000010b8d20c456a:T=1746629716:RT=1746779601:S=ALNI_Man7GEiGSfz4CbBR0Z5m2myvOv6Ig; __eoi=ID=0ecb8dd43c3a89f0:T=1746629716:RT=1746779602:S=AA-AfjbUuUTAiH0L37z1mAGUM5g0; _ga_QX0Q50ZC9P=GS2.1.s1746777027$o6$g1$t1746779610$j51$l0$h0; datadome=lI15Ea2XfCtGPKeKrC~tuWyqrAx6SlT6LNVH0pOH9q6RVogsnxBw0_mOj6OXpMv6ibWr2os6i5BSrgWL_FYxO5UxdmYVqZYdkYt4tgRXXto2Z2_DWfsunc_GLY9q2HZT; __vt=wmHuiTzM4bFEj0FUABQCT24E-H_BQo6gx1APGQJPtzVSDq_N0cSaiaKKBEwXyZzygz5CKQ-SSlXiW2hEvXE1tYjqsUxt_CQvB3FVVpLyrcJ6NUFqooCr0JcmXnTF_5e7Ri2CsngMgNxqIJYkc2uL9QG0HVY'  # TripAdvisor 返回的 Cookie


def fetch_list_pages(pages=1):
    """
    抓取餐厅列表页
    :param pages: 要抓取的页数，每页 30 条
    :return: 包含名称和详情链接的字典列表
    """
    restaurants = []  # 用于存储所有餐厅信息
    for page in range(pages):
        offset = page * 30  # 计算分页偏移量
        url = f"{BASE_URL}/FindRestaurants?geo=308272&offset={offset}&sort=FEATURED&broadened=false"  # 列表页 URL
        print(f"[列表] 爬第 {page+1} 页：{url}")  # 打印当前抓取页信息
        r = requests.get(url, headers=headers, timeout=15)  # 发送 GET 请求
        r.raise_for_status()  # 若状态码不是 200，则抛出异常
        tree = etree.HTML(r.text)  # 将返回的 HTML 文本解析成 etree
        
        # 通过 XPath 定位列表项 div
        divs = tree.xpath("//div[contains(@class,'mtnKn') and contains(@class,'ngXxk')]")
        for d in divs:
            raw = d.xpath("string(.)").strip()  # 提取节点文本并去除两端空白
            name = re.sub(r'^\d+\.\s*', '', raw)  # 去掉前缀序号
            href = d.xpath("ancestor::a[1]/@href")  # 向上查找最近的 a 标签链接
            if href:
                restaurants.append({
                    "名称": name,  # 餐厅名称
                    "链接": BASE_URL + href[0]  # 详情页完整 URL
                })
        # 随机睡眠 5~8 秒，防止请求过于频繁导致封禁
        time.sleep(5 + random.random() * 3)
    return restaurants  # 返回抓取到的餐厅列表


def fetch_detail(html_text):
    """
    从详情页 HTML 文本中提取信息
    :param html_text: 详情页的 HTML 字符串
    :return: 包含评分、评价数、类别、人均、地址等信息的字典
    """
    tree = etree.HTML(html_text)  # 解析 HTML
    detail = {}  # 存储解析结果
    
    # 提取评分（泡泡评分）
    rating = tree.xpath("//div[@data-automation='bubbleRatingValue']/text()")
    detail['评分'] = rating[0].strip() if rating else None  # 取第一个并去空，否则 None
    
    # 提取评价数，支持千分位格式
    reviews = tree.xpath("//div[@data-automation='bubbleReviewCount']//text()")
    if reviews:
        # 合并所有文本，并匹配数字及逗号
        m = re.search(r'([\d,]+)', "".join(reviews))
        detail['评价数'] = m.group(1).replace(',', '') if m else None  # 去掉逗号
    else:
        detail['评价数'] = None
    
    # 提取类别和人均消费
    items = tree.xpath("//span[contains(@class,'HUMGB') and contains(@class,'cPbcf')]//a/span/text()")
    if items:
        last = items[-1].strip()  # 取最后一个条目
        if '¥' in last:
            detail['人均'] = last  # 若包含人民币符号，则为人均消费
            # 类别为其余条目
            detail['类别'] = ",".join(i.strip() for i in items[:-1] if i.strip())
        else:
            detail['人均'] = None
            detail['类别'] = ",".join(i.strip() for i in items if i.strip())
    else:
        detail['人均'] = None
        detail['类别'] = None
    
    # 提取地址，尝试多种备选 XPath
    addr = tree.xpath("//span[@data-automation='restaurantsMapLinkOnName']/text()")
    if not addr:
        addr = tree.xpath("//button[@data-automation='restaurantsMapLinkOnName']//span/text()")
    detail['地址'] = addr[0].strip() if addr else None
    
    return detail  # 返回详情信息字典


def main():
    """
    主函数：先抓取列表页，再抓取详情页，最后保存为 CSV
    :return: 包含所有抓取数据的 DataFrame
    """
    # 1) 抓列表页（默认 100 页）
    lst = fetch_list_pages(pages=100)
    df_list = pd.DataFrame(lst)  # 转成 DataFrame 方便迭代

    # 2) 抓详情页
    records = []  # 存储所有详情记录
    for idx, row in df_list.iterrows():
        print(f"[详情] {idx+1}/{len(df_list)} {row['名称']}")  # 打印当前进度
        try:
            r = requests.get(row['链接'], headers=headers, timeout=15)  # 请求详情页
            r.raise_for_status()
            info = fetch_detail(r.text)  # 解析详情
        except Exception as e:
            print(f"  详情页失败：{e}")  # 出错时打印异常
            # 出错时构造空值
            info = {'评分': None, '评价数': None, '类别': None, '人均': None, '地址': None}
        # 合并名称与链接
        info.update({"名称": row['名称'], "链接": row['链接']})
        records.append(info)  # 加入记录列表
        # 随机睡眠 2~4 秒
        time.sleep(2 + random.random() * 2)
    
    # 3) 合并并保存为 CSV
    result = pd.DataFrame(records, columns=['名称','链接','评分','评价数','类别','人均','地址'])
    result.to_csv("tripadvisor_shanghai.csv", index=False, encoding="utf-8-sig")  # 保存文件
    print("已保存到 tripadvisor_shanghai.csv，共抓取条数：", len(result))
    return result
df = main()


[列表] 爬第 1 页：https://cn.tripadvisor.com/FindRestaurants?geo=308272&offset=0&sort=FEATURED&broadened=false
[列表] 爬第 2 页：https://cn.tripadvisor.com/FindRestaurants?geo=308272&offset=30&sort=FEATURED&broadened=false
[列表] 爬第 3 页：https://cn.tripadvisor.com/FindRestaurants?geo=308272&offset=60&sort=FEATURED&broadened=false
[列表] 爬第 4 页：https://cn.tripadvisor.com/FindRestaurants?geo=308272&offset=90&sort=FEATURED&broadened=false
[列表] 爬第 5 页：https://cn.tripadvisor.com/FindRestaurants?geo=308272&offset=120&sort=FEATURED&broadened=false
[列表] 爬第 6 页：https://cn.tripadvisor.com/FindRestaurants?geo=308272&offset=150&sort=FEATURED&broadened=false
[列表] 爬第 7 页：https://cn.tripadvisor.com/FindRestaurants?geo=308272&offset=180&sort=FEATURED&broadened=false
[列表] 爬第 8 页：https://cn.tripadvisor.com/FindRestaurants?geo=308272&offset=210&sort=FEATURED&broadened=false
[列表] 爬第 9 页：https://cn.tripadvisor.com/FindRestaurants?geo=308272&offset=240&sort=FEATURED&broadened=false
[列表] 爬第 10 页：https://cn.tripadvisor.com/Fi

KeyboardInterrupt: 

<p style="text-indent:2em;">
<strong>第二阶段：</strong> 经过一次失败后，与该网页逐渐"熟悉"。但是我们天真认为该网页是按照访问次数来封的，觉得之前试爬取的时候浪费了很多访问机会。于是第二天没有做很多修改直接开始爬，结果不出意料IP马上被封了，后面总结出是更具访问频率和其他方面来封的。由于<span style="color: red;">时间紧迫</span>，只好请代理IP来帮助（小氪一点），并且经过讨论，我们不再天真，加上了边跑边存的机制，并且清楚还会出现访问失败的情况，于是用另外一个visited_links.txt文件来存取访问过的餐厅的网址，方便后续对失败的网页再次访问。由于本次采用的是串行爬取，并且为了模拟真人，每次爬取休息时间也较长，所以<strong><span style="color: blue;">爬了6个小时以上</span></strong>。但是，<span style="color: red;">即使这样😢</span>，我们仍只爬到了1939条数据（保存在tripadvisor_shanghai.csv中），根据输出显示与AI咨询，总结原因如下：

1.由于财力有限，没有用很好的IP，导致越爬到后面，有限的代理IP越容易被封禁，这也符合输出显示中的现象。

2.网页通过JavaScript部分采用了动态渲染和更新，而requests + lxml只能能拿到初始化好的静态的HTML中的内容，导致很多空的爬取。借助AI，也尝试了利用Selenium来模拟真实的浏览器访问，最终以访问速度慢以及效果不佳而放弃。

3.列表页爬取失败直接缺失30条，总共有30条列表页访问失败；其次访问详情页的时候也会失败，最终只获得1939条数据，并且这些数据有些残缺。  
此处**失败**包括访问页面时的403问题（IP被封）和超出了提前设定好的HTML获取时间15s。
</p>
<p style="text-indent:2em;">代码如下：</p>

In [ ]:
import requests  # 用于发送 HTTP 请求
from lxml import etree  # 用于解析 HTML
import pandas as pd  # 用于数据处理与保存
import time  # 用于控制爬取时间间隔
import random  # 用于生成随机延时，防止被封禁
import re  # 用于正则表达式匹配
import os  # 用于文件操作

BASE_URL = "https://cn.tripadvisor.com"  # TripAdvisor 中国站基础 URL

# 快代理配置
tunnel = "t828.kdltpspro.com:15818"  # 代理地址:端口
username = "t14703884308185"  # 代理用户名
password = "a1o7xj86"  # 代理密码
proxies = {
    "http": f"http://{username}:{password}@{tunnel}",  # HTTP 代理
    "https": f"http://{username}:{password}@{tunnel}"  # HTTPS 代理
}

# HTTP 请求头，模拟浏览器行为
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 '
                  '(KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
    'Accept-Language': 'zh-CN,zh;q=0.9',
    'Referer': 'https://cn.tripadvisor.com/',
    'Connection': 'keep-alive',
    'Upgrade-Insecure-Requests': '1',
    'Accept-Encoding': 'gzip, deflate, br',
}

def safe_print(text):
    """安全打印，捕获 Unicode 编码错误"""
    try:
        print(text)
    except UnicodeEncodeError:
        # 遇到无法编码字符时忽略后打印
        print(text.encode('utf-8', errors='ignore').decode('utf-8'))


def load_visited_links(file="visited_links.txt"):
    """加载已访问链接，防止重复爬取"""
    if os.path.exists(file):
        with open(file, "r", encoding="utf-8") as f:
            return set(f.read().splitlines())  # 返回存储的链接集合
    return set()  # 若文件不存在返回空集合


def save_visited_links(visited_links, file="visited_links.txt"):
    """将新访问的链接追加写入文件"""
    with open(file, "a", encoding="utf-8") as f:
        for link in visited_links:
            f.write(f"{link}\n")  # 每行写入一个链接


def fetch_list_pages(pages=1):
    """抓取列表页，提取餐厅名称和链接"""
    restaurants = []
    for page in range(pages):
        offset = page * 30  # 每页 30 条记录
        url = f"{BASE_URL}/FindRestaurants?geo=308272&offset={offset}&sort=FEATURED&broadened=false"
        safe_print(f"[列表] 爬第 {page+1} 页：{url}")
        try:
            # 请求列表页
            r = requests.get(url, headers=headers, proxies=proxies, timeout=15)
            r.raise_for_status()
        except Exception as e:
            safe_print(f"  列表页失败：{e}")
            continue

        tree = etree.HTML(r.text)  # 解析 HTML
        # 定位包含餐厅信息的 div
        divs = tree.xpath("//div[contains(@class,'mtnKn') and contains(@class,'ngXxk')]")
        for d in divs:
            raw = d.xpath("string(.)").strip()  # 获取文本并去除空白
            name = re.sub(r'^\d+\.\s*', '', raw)  # 去掉序号前缀
            href = d.xpath("ancestor::a[1]/@href")  # 向上查找链接
            if href:
                restaurants.append({"名称": name, "链接": BASE_URL + href[0]})
        time.sleep(4 + random.random() * 2)  # 随机延时 2-4 秒
    return restaurants


def fetch_detail(html_text):
    """解析详情页，提取评分、评价数、类别、人均、地址"""
    tree = etree.HTML(html_text)
    detail = {}
    # 提取评分
    rating = tree.xpath("//div[@data-automation='bubbleRatingValue']/text()")
    detail['评分'] = rating[0].strip() if rating else None

    # 提取评价数
    reviews = tree.xpath("//div[@data-automation='bubbleReviewCount']//text()")
    if reviews:
        m = re.search(r'([\d,]+)', "".join(reviews))
        detail['评价数'] = m.group(1).replace(',', '') if m else None
    else:
        detail['评价数'] = None

    # 提取类别和人均
    items = tree.xpath("//span[contains(@class,'HUMGB') and contains(@class,'cPbcf')]//a/span/text()")
    if items:
        last = items[-1].strip()
        if '¥' in last:
            detail['人均'] = last
            detail['类别'] = ",".join(i.strip() for i in items[:-1] if i.strip())
        else:
            detail['人均'] = None
            detail['类别'] = ",".join(i.strip() for i in items if i.strip())
    else:
        detail['人均'] = None
        detail['类别'] = None

    # 提取地址
    addr = tree.xpath("//span[@data-automation='restaurantsMapLinkOnName']/text()")
    if not addr:
        addr = tree.xpath("//button[@data-automation='restaurantsMapLinkOnName']//span/text()")
    detail['地址'] = addr[0].strip() if addr else None
    return detail


def main(pages=100):
    output_file = "tripadvisor_shanghai.csv"  # 输出 CSV 文件名
    visited_links = load_visited_links()  # 加载已访问链接
    all_data = []  # 存储本次爬取的数据

    # 如果输出文件不存在，则写入表头
    if not os.path.exists(output_file):
        pd.DataFrame(columns=['名称', '链接', '评分', '评价数', '类别', '人均', '地址'])\
          .to_csv(output_file, index=False, encoding="utf-8-sig")

    # 抓取列表页数据
    lst = fetch_list_pages(pages)

    new_visited_links = set()  # 本次新增访问链接
    # 逐条抓取详情页并写入 CSV
    for idx, row in enumerate(lst):
        if row['链接'] in visited_links:
            safe_print(f"  跳过已访问: {row['链接']}")
            continue
        safe_print(f"[详情] {idx+1}/{len(lst)} {row['名称']}")
        try:
            r = requests.get(row['链接'], headers=headers, proxies=proxies, timeout=15)
            r.raise_for_status()
            info = fetch_detail(r.text)
        except Exception as e:
            safe_print(f"  详情页失败：{e}")
            info = {'评分': None, '评价数': None, '类别': None, '人均': None, '地址': None}
        info.update({"名称": row['名称'], "链接": row['链接']})
        # 追加写入 CSV
        pd.DataFrame([info]).to_csv(output_file, mode="a", header=False, index=False, encoding="utf-8-sig")
        new_visited_links.add(row['链接'])  # 记录新访问链接
        all_data.append(info)  # 收集数据
        time.sleep(5 + random.random() * 3)  # 随机延时

    # 保存本次访问链接
    save_visited_links(new_visited_links)
    safe_print(f"完成，结果保存在 {output_file}")
    return pd.DataFrame(all_data)

# 调用入口
df = main(100)  # 默认抓取 100 页

<p style="text-indent:2em;">
<strong>第三阶段：</strong><span style="color: blue;">最后的倔强</span>阶段。本着不放弃最初设定的3000数据的梦想😥，首先检查了阶段二爬下来的1939条数据与相应的网页原代码，发现有的数据为空是因为<strong>有的网页本身HTML不同，有的网页本来就不包括人均、类别等数据</strong>，于是我们对xpath路径做了兼容补充。然后又尝试了多种方法，比如，根据第二阶段保存的餐厅链接去访问访问失败了的网页的数据，但最终都以各种原因失败而告终，并且还爱在不断的尝试中<span style="color: red;">不小心弄丢了</span>阶段二保存了的visited_links.txt文件，这也成为压死骆驼的最后一根稻草😫，导致无法再直接重新爬取失败了的网页数据了。然后又尝试了直接重新爬取，比较可行的是下面的方案：除了补充xpath路径外，还将原来的串行爬取改成了并行，极大程度上提升了爬行速度，但是被封的概率也极大提升了，不管采用几线并行，中间休息多少秒都无法再爬取更多有效信息了。
</p>

<p style="text-indent:2em;">另外，本次尝试还了解到一个<strong><span style="color: blue;">疑惑冷知识:</strong>  尝试了免费体验的代理IP，各方面性能比我们之前小氪的代理IP要好，并且查看了续费价格也更贵，但是实际效果为什么就是比我们之前的差（起初还以为如获至宝😢）</p>
<p style="text-indent:2em;">代码如下：</p>

In [ ]:
import requests  # 用于发送 HTTP 请求
from lxml import etree  # 用于解析 HTML
import pandas as pd  # 用于数据处理与保存
import time  # 用于控制爬取间隔
import random  # 用于生成随机延时
import re  # 用于正则匹配
import os  # 用于文件和目录操作
from requests.adapters import HTTPAdapter  # 用于设置重试策略
from urllib3.util.retry import Retry  # 重试策略配置
from concurrent.futures import ThreadPoolExecutor, as_completed  # 并发抓取

# ----- 基础常量配置 -----
BASE_URL = "https://cn.tripadvisor.com"  # TripAdvisor 中国站基础 URL
TUNNEL = "s659.kdltps.com:15818"  # 代理服务器地址:端口
USERNAME = "t14704858604183"  # 代理用户名
PASSWORD = "jz5qpwel"  # 代理密码
# 构造代理字典，requests 会自动识别
PROXIES = {
    "http": f"http://{USERNAME}:{PASSWORD}@{TUNNEL}",
    "https": f"http://{USERNAME}:{PASSWORD}@{TUNNEL}"
}
# HTTP 请求头，伪装浏览器
HEADERS = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 '
                  '(KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
    'Accept-Language': 'zh-CN,zh;q=0.9',
    'Referer': BASE_URL,
    'Connection': 'keep-alive',
    'Upgrade-Insecure-Requests': '1',
    'Accept-Encoding': 'gzip, deflate, br',
}

# ----- 输出与进度文件 -----
OUTPUT_FILE = "tripadvisor_shanghai.csv"  # 最终结果 CSV
FAILURES_DIR = "failures"  # 存放失败详情的文件夹
FAILURES_FILE = os.path.join(FAILURES_DIR, "failed_details.csv")  # 失败详情 CSV
PROGRESS_FILE = "fetched_pages.txt"  # 存放已抓取列表页页码
COLUMNS = ['名称', '链接', '评分', '评价数', '类别', '人均', '地址']  # CSV 列顺序
TOTAL_PAGES = 100  # 列表页总数
MAX_DETAILS_PER_RUN = 90  # 本次运行最多抓取详情数

# ----- 初始化文件与目录 -----
os.makedirs(FAILURES_DIR, exist_ok=True)  # 确保失败目录存在
# 若输出文件不存在，则写入表头
if not os.path.exists(OUTPUT_FILE):
    pd.DataFrame(columns=COLUMNS).to_csv(OUTPUT_FILE, index=False, encoding='utf-8-sig')
# 若失败文件不存在，则写入表头
if not os.path.exists(FAILURES_FILE):
    pd.DataFrame(columns=['名称','链接','错误信息']).to_csv(FAILURES_FILE, index=False, encoding='utf-8-sig')

# ----- Session 初始化（带重试与代理） -----
session = requests.Session()
# 配置重试策略：最多重试 5 次，针对特定状态码
retry = Retry(
    total=5,
    backoff_factor=0.3,
    status_forcelist=[403, 500, 502, 503, 504],
    allowed_methods=frozenset(["GET"])
)
# 将重试策略安装到 session
session.mount('http://', HTTPAdapter(max_retries=retry))
session.mount('https://', HTTPAdapter(max_retries=retry))
# 更新代理与请求头到 session
session.proxies.update(PROXIES)
session.headers.update(HEADERS)


def safe_sleep(min_sec=2, max_sec=4):
    """随机睡眠，避免过快请求触发反爬"""
    time.sleep(min_sec + random.random() * (max_sec - min_sec))


def fetch_list_pages(pages=TOTAL_PAGES, max_retries=3):
    """
    抓取列表页，提取餐厅名和详情链接，维护进度与去重
    :param pages: 总页数
    :param max_retries: 单页最大重试次数
    :return: 新的餐厅链接列表
    """
    restaurants = []  # 存储本次需抓取的餐厅
    fetched_pages = set()
    # 读取已抓取页码，避免重复
    if os.path.exists(PROGRESS_FILE):
        with open(PROGRESS_FILE, 'r') as f:
            fetched_pages = set(int(line.strip()) for line in f if line.strip().isdigit())
    # 读取已写入 OUTPUT_FILE 的链接，避免重复抓取详情
    if os.path.exists(OUTPUT_FILE):
        fetched_df = pd.read_csv(OUTPUT_FILE, usecols=["链接"], encoding='utf-8-sig')
        fetched_urls = set(fetched_df['链接'].dropna().unique())
    else:
        fetched_urls = set()

    # 遍历每个列表页
    for page in range(pages):
        if page in fetched_pages:
            print(f"[跳过] 第 {page+1} 页已爬取")
            continue

        offset = page * 30
        url = (f"{BASE_URL}/FindRestaurants?geo=308272"
               f"&offset={offset}&sort=FEATURED&broadened=false")
        print(f"[列表] 第 {page+1}/{pages} 页：{url}")

        # 重试机制
        for attempt in range(1, max_retries+1):
            try:
                r = session.get(url, timeout=15)  # 发请求
                r.raise_for_status()
                tree = etree.HTML(r.text)
                # 定位包含餐厅信息的节点
                divs = tree.xpath("//div[contains(@class,'mtnKn') and contains(@class,'ngXxk')]")
                for d in divs:
                    raw = d.xpath("string(.)").strip()
                    name = re.sub(r'^\d+\.\s*', '', raw)  # 去序号
                    href = d.xpath("ancestor::a[1]/@href")
                    if href:
                        full_url = BASE_URL + href[0]
                        # 若未在已抓取列表中，则添加
                        if full_url not in fetched_urls:
                            restaurants.append({"名称": name, "链接": full_url})
                            # 达到上限则提前返回
                            if len(restaurants) >= MAX_DETAILS_PER_RUN:
                                print(f"✅ 达到上限 {MAX_DETAILS_PER_RUN} 条，提前停止")
                                with open(PROGRESS_FILE, 'a') as f:
                                    f.write(f"{page}\n")
                                return restaurants
                # 当前页成功抓取后，记录页码
                with open(PROGRESS_FILE, 'a') as f:
                    f.write(f"{page}\n")
                break

            except Exception as e:
                print(f"  [列表页重试 {attempt}/{max_retries}] 失败: {e}")
                safe_sleep(3,5)
        else:
            print(f"  ❌ 列表页最终失败，跳过：{url}")

        safe_sleep(2,4)

    return restaurants


def fetch_detail(row, max_retries=3):
    """
    抓取单个餐厅的详情页并解析，失败记录到文件
    :param row: 包含 '链接' 和 '名称' 的 dict
    :param max_retries: 最大重试次数
    :return: 含详情字段的 dict
    """
    url, name = row['链接'], row['名称']
    detail = {}
    for attempt in range(1, max_retries+1):
        try:
            print(f"[详情] {name} - 尝试 {attempt}/{max_retries}")
            r = session.get(url, timeout=15)
            r.raise_for_status()
            tree = etree.HTML(r.text)
            # 提取评分
            rt = tree.xpath("//div[@data-automation='bubbleRatingValue']/text()")
            detail['评分'] = rt[0].strip() if rt else '未标注'
            # 提取评价数
            rev = tree.xpath("//div[@data-automation='bubbleReviewCount']//text()")
            detail['评价数'] = (
                re.search(r'([\d,]+)', "".join(rev)).group(1).replace(',','')
                if rev and re.search(r'([\d,]+)', "".join(rev)) else '未标注'
            )
            # 提取类别和人均
            items = tree.xpath(
                "//span[contains(@class,'HUMGB') and contains(@class,'cPbcf')]//a/span/text()"
            )
            if items:
                last = items[-1].strip()
                if '¥' in last:
                    detail['人均'] = last
                    detail['类别'] = ",".join(i.strip() for i in items[:-1] if i.strip()) or '未标注'
                else:
                    detail['人均'] = '未标注'
                    detail['类别'] = ",".join(i.strip() for i in items if i.strip()) or '未标注'
            else:
                detail['类别'] = detail['人均'] = '未标注'
            # 提取地址
            addr = tree.xpath("//span[@data-automation='restaurantsMapLinkOnName']/text()")
            if not addr:
                addr = tree.xpath("//button[@data-automation='restaurantsMapLinkOnName']//span/text()")
            detail['地址'] = addr[0].strip() if addr else '未标注'
            break

        except requests.exceptions.HTTPError as e:
            code = e.response.status_code
            if code == 403 and attempt < max_retries:
                print(f"  [403重试 {attempt}/{max_retries}]")
                safe_sleep(3,5)
                continue
            err = f"HTTP{code}"
        except Exception as e:
            err = str(e)
        # 失败时记录到失败文件
        detail = dict.fromkeys(['评分','评价数','类别','人均','地址'], '未标注')
        with open(FAILURES_FILE, 'a', encoding='utf-8-sig', newline='') as f:
            pd.DataFrame([{'名称':name,'链接':url,'错误信息':err}]).to_csv(
                f, header=False, index=False, encoding='utf-8-sig'
            )
        print(f"  ❌ 最终失败：{name} - {err}")
        break

    safe_sleep(2,4)
    return {**row, **detail}  # 合并基本信息和详情


def main():
    # 读取已抓取过的链接，用于去重详情爬取
    if os.path.exists(OUTPUT_FILE):
        fetched_df = pd.read_csv(OUTPUT_FILE, encoding='utf-8-sig', usecols=['链接'])
        fetched_urls = set(fetched_df['链接'].dropna().unique())
    else:
        fetched_urls = set()

    # 获取列表页链接
    lst = fetch_list_pages()
    # 过滤掉已抓过的链接
    lst = [r for r in lst if r['链接'] not in fetched_urls]
    print(f"实际需要爬取详情页：{len(lst)} 条")

    if not lst:
        print("🎉 没有新链接需要抓取，本轮结束")
        return

    # 并发抓取详情页，最多 5 个线程
    completed = 0
    with ThreadPoolExecutor(max_workers=5) as executor:
        futures = {executor.submit(fetch_detail, r): r for r in lst}
        for future in as_completed(futures):
            info = future.result()
            completed += 1
            print(f"[进度] {completed}/{len(lst)}")
            # 将每条结果追加写入 OUTPUT_FILE
            pd.DataFrame([info])[COLUMNS].to_csv(
                OUTPUT_FILE, mode='a', header=False, index=False, encoding='utf-8-sig'
            )
            # 每 30 条结果暂停 60 秒
            if completed % 30 == 0:
                print("  —— 长休 60 秒 ——")
                time.sleep(60)

    # 去重，确保链接唯一
    df = pd.read_csv(OUTPUT_FILE, encoding='utf-8-sig', dtype=str)
    df.drop_duplicates(subset=['链接'], keep='first', inplace=True)
    df.to_csv(OUTPUT_FILE, index=False, encoding='utf-8-sig')
    print(f"✅ 本轮完成：新增 {completed} 条，累计 {len(df)} 条；失败见 {FAILURES_FILE}")

if __name__ == "__main__":
    main()


[列表] 第 1/100 页：https://cn.tripadvisor.com/FindRestaurants?geo=308272&offset=0&sort=FEATURED&broadened=false
[列表] 第 2/100 页：https://cn.tripadvisor.com/FindRestaurants?geo=308272&offset=30&sort=FEATURED&broadened=false
[列表] 第 3/100 页：https://cn.tripadvisor.com/FindRestaurants?geo=308272&offset=60&sort=FEATURED&broadened=false
✅ 达到上限 90 条，提前停止
实际需要爬取详情页：90 条
[详情] 上海佳家汤包(黄河路店) - 尝试 1/3
[详情] 星巴克臻选上海烘焙工坊 - 尝试 1/3
[详情] 小杨生煎馆(吴江路店) - 尝试 1/3
[详情] ROOF (The Shanghai EDITION) - 尝试 1/3
[详情] 金轩(上海浦东丽思卡尔顿酒店) - 尝试 1/3
[详情] 上海小杨生煎(黄河路店) - 尝试 1/3
[进度] 1/90
[详情] 小杨生煎(宁波路店) - 尝试 1/3
[进度] 2/90
[详情] Flair Rooftop - 尝试 1/3[进度] 3/90

[详情] 莉莲蛋挞(四季坊店) - 尝试 1/3
[进度] 4/90
[详情] Scena di Angelo 意大利餐厅 - 尝试 1/3[进度] 5/90

[详情] Yone餐厅酒吧 (上海艾迪逊酒店) - 尝试 1/3[进度] 6/90

[详情] 怡咖啡 - 尝试 1/3
[进度] 7/90
[详情] 上海香溢培客(安福路店) - 尝试 1/3
[进度] 8/90
[详情] Hakkasan - 尝试 1/3
[进度] 9/90
[详情] 芝乐坊餐厅 - 尝试 1/3
[进度] 10/90
[详情] Aura酒廊及爵士酒吧(上海浦东丽思卡尔顿酒店) - 尝试 1/3[进度] 11/90

[详情] The Fellas - 尝试 1/3[进度] 12/90

[详情] 花马天堂云南餐厅(外滩店) - 尝试 1/3[进度] 13/90

[详情

KeyboardInterrupt: 

### <span style="color: blue;">3.简单数据处理</span>
<p style="text-indent:2em;">
通过<strong>"过程介绍"</strong>中的分析，我们最终还是采用了阶段2中获取的1939条数据，并且进行了简单处理：首先除去所有不包括地址、评分、评价数中任意一个的餐厅，其次在剩下的餐厅中没有人均和类别的都标注上“未注明”，最终获得了1504条数据，保存在tripadvisor_shanghai_cleaned.csv.xlsx中。
</p>

In [ ]:
import pandas as pd

# 1. 读取原始 CSV
df = pd.read_excel('tripadvisor_shanghai_fixed.csv.xlsx')

# 2. 删除 “地址”、“评分”、“评价数” 任一列存在缺失值的行
df = df.dropna(subset=['地址', '评分', '评价数'])

# 3. 对 “类别” 和 “人均” 列的缺失值填充 “未注明”
df['类别'] = df['类别'].fillna('未注明')
df['人均'] = df['人均'].fillna('未注明')

# 4. 保存清洗后的结果
df.to_excel('tripadvisor_shanghai_fixed_cleaned.csv.xlsx', index=False)